In [20]:
import numpy as np
import pandas as pd


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
file='/content/drive/MyDrive/Colab Notebooks/train_v2.csv'

In [23]:
df_raw=pd.read_csv(file,na_values='NA')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (135,204,274,417) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# New section

In [24]:
df_raw.info()
print(df_raw.describe(include='all'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 771 entries, id to loss
dtypes: float64(653), int64(99), object(19)
memory usage: 620.4+ MB
                   id             f1  ...           f778           loss
count   105471.000000  105471.000000  ...  105471.000000  105471.000000
unique            NaN            NaN  ...            NaN            NaN
top               NaN            NaN  ...            NaN            NaN
freq              NaN            NaN  ...            NaN            NaN
mean     52736.000000     134.603171  ...     175.951589       0.799585
std      30446.999458      14.725467  ...     298.294043       4.321120
min          1.000000     103.000000  ...       2.000000       0.000000
25%      26368.500000     124.000000  ...      19.000000       0.000000
50%      52736.000000     129.000000  ...      40.000000       0.000000
75%      79103.500000     148.000000  ...     104.000000       0.000000
max     105471.000000     176

In [25]:
column_to_drop=[]
#identify and drop non-numerical features
for c in df_raw.columns:
    if(df_raw[c].dtypes==object):
        column_to_drop.append(c)
print("Column to drop:",column_to_drop)
df_raw=df_raw.drop(column_to_drop,axis=1)
df_raw.info()


Column to drop: ['f137', 'f138', 'f206', 'f207', 'f276', 'f277', 'f338', 'f390', 'f391', 'f419', 'f420', 'f469', 'f472', 'f534', 'f537', 'f626', 'f627', 'f695', 'f698']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 752 entries, id to loss
dtypes: float64(653), int64(99)
memory usage: 605.1 MB


In [26]:
#handling missing data
column_with_missing_value=dict()
for c in df_raw.columns:
    if df_raw[c].isnull().sum() > 0:
        column_with_missing_value[c]=df_raw[c].isnull().sum()
        
import pprint
pp=pprint.PrettyPrinter(indent=4)
print("Columns with missing values:")
pp.pprint(column_with_missing_value)



Columns with missing values:
{   'f100': 1704,
    'f101': 1704,
    'f102': 335,
    'f103': 335,
    'f104': 182,
    'f105': 182,
    'f106': 182,
    'f107': 182,
    'f108': 182,
    'f109': 182,
    'f110': 1491,
    'f111': 1491,
    'f112': 335,
    'f113': 335,
    'f114': 182,
    'f115': 182,
    'f116': 182,
    'f117': 182,
    'f118': 182,
    'f119': 182,
    'f120': 1121,
    'f121': 1121,
    'f122': 335,
    'f123': 335,
    'f124': 182,
    'f125': 182,
    'f126': 182,
    'f127': 182,
    'f128': 182,
    'f129': 182,
    'f130': 808,
    'f131': 808,
    'f132': 335,
    'f133': 335,
    'f134': 182,
    'f135': 182,
    'f136': 182,
    'f139': 182,
    'f14': 100,
    'f140': 682,
    'f141': 682,
    'f142': 2561,
    'f143': 2561,
    'f144': 1291,
    'f145': 1291,
    'f146': 1291,
    'f147': 1291,
    'f148': 1291,
    'f149': 2859,
    'f15': 48,
    'f150': 2859,
    'f151': 2561,
    'f152': 2561,
    'f153': 1291,
    'f154': 1291,
    'f155': 1291,
  

In [27]:
from sklearn.impute import SimpleImputer
from sklearn_pandas import gen_features
from sklearn_pandas import DataFrameMapper

In [28]:
column_to_impute=[]
for key, value in column_with_missing_value.items():
    column_to_impute.append([key])
feature_def=gen_features(
    columns=column_to_impute,
    classes=[{'class':SimpleImputer,'strategy':'most_frequent'}]
)
mapper=DataFrameMapper(feature_def, default=None,input_df=True,df_out=True)
df_imputed=mapper.fit_transform(df_raw)
df_imputed.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 752 entries, f7 to loss
dtypes: float64(653), int64(99)
memory usage: 605.1 MB


In [29]:
column_with_missing_values=dict()
for c in df_imputed.columns:
    if df_imputed[c].isnull().sum()>0:
        colums_with_missing_values[c]=df_imputed[c].isnull().sum()
        
pp=pprint.PrettyPrinter(indent=4)
print("Columns with missing values:")
pp.pprint(column_with_missing_values)


Columns with missing values:
{}


In [34]:
#one hot encoding for categorical features
categorical_threshold=100
column_single_value=[]
column_categorical=[]
column_continuous=[]
for c in df_raw.columns:
  if (c=='id'or c=='loss'):
    continue
  if (df_raw[c].dtypes=='int64' and len(df_raw.loc[:,c].unique())<=categorical_threshold):
    print("Column \""+c +"\" has unique values:")
    print(df_raw.loc[:,c].unique())
    if (len(df_raw.loc[:,c].unique())==1):
      column_single_value.append(c)
    else:
      column_categorical.append([c])
  else:
    column_continuous.append([c])
del df_raw

print("Categorial columns:",column_categorical)
print("Single valued columns:",column_single_value)

  

Column "f1" has unique values:
[126 121 134 109 128 127 115 120 130 119 116 123 125 114 129 117 124 122
 139 144 142 137 154 150 147 156 157 146 153 135 138 118 113 133 110 111
 136 132 108 106 152 145 149 155 163 166 165 160 148 159 151 162 158 131
 143 161 164 140 141 173 167 112 168 107 170 105 171 103 104 169 174 176
 172 175]
Column "f2" has unique values:
[10  9  8  6  7  4  2  3  1 11]
Column "f4" has unique values:
[1100 2900 1300 1800 2200 3800 2600 1600 3500 1900 2700 4800 4200 3600
 3700 1500 2400 4300 5400 2000 3100 5500 5000 4400 4700 2300 5100 5300
 1200 3900 6200 5900 5700 4100 4000 4500 1700 2500 6600 3300 2100 3000
 5800 3200 2800 4600 5600 7200 3400 1400 5200 6100 6800 7000 6500 6300
 6400 7300 6700 7400 7900]
Column "f5" has unique values:
[ 3  4 16 10  2  7 15  1 17 13]
Column "f13" has unique values:
[ 7  6 14 13 15  8 10 11  4 16 12  5  9  3 17 18 20 19 25 29 28 27 23 26
 21 24 22 32  2 40 30]
Column "f33" has unique values:
[0]
Column "f34" has unique values:
[0]

In [35]:
#drop single value column
df_imputed=df_imputed.drop(column_single_value,axis=1)
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 742 entries, f7 to loss
dtypes: float64(653), int64(89)
memory usage: 597.1 MB


In [36]:
#Replacing categorical features with on-hot encoding
from sklearn.preprocessing import OneHotEncoder
feature_def=gen_features(
    columns=column_categorical,
    classes=[OneHotEncoder]
)
mapper = DataFrameMapper(feature_def,default=None, input_df=True, df_out=True)
df_imputed=mapper.fit_transform(df_imputed)
df_imputed.info()
print(df_imputed.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 1790 entries, f1_x0_103 to loss
dtypes: float64(1736), int64(54)
memory usage: 1.4 GB
           f1_x0_103      f1_x0_104  ...           f773           loss
count  105471.000000  105471.000000  ...  105471.000000  105471.000000
mean        0.000284       0.000076  ...       3.273059       0.799585
std         0.016863       0.008709  ...       3.766746       4.321120
min         0.000000       0.000000  ...       0.000000       0.000000
25%         0.000000       0.000000  ...       0.740000       0.000000
50%         0.000000       0.000000  ...       1.990000       0.000000
75%         0.000000       0.000000  ...       4.440000       0.000000
max         1.000000       1.000000  ...      34.040000     100.000000

[8 rows x 1790 columns]


In [ ]:
df_imputed.to_csv('train_v2_preprocessed.csv')

In [37]:
#normalizing continuous features
from sklearn.preprocessing import StandardScaler
feature_def=gen_features(
    columns=column_continuous,
    classes=[StandardScaler]
)

mapper= DataFrameMapper(feature_def,default=None, input_df=True, df_out=True)
df_imputed_normalized=mapper.fit_transform(df_imputed)
df_imputed_normalized.info()
print(df_imputed_normalized.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 1790 entries, f3 to loss
dtypes: float64(1788), int64(2)
memory usage: 1.4 GB
                 f3            f6  ...             id           loss
count  1.054710e+05  1.054710e+05  ...  105471.000000  105471.000000
mean   5.622168e-17 -2.554207e-15  ...   52736.000000       0.799585
std    1.000005e+00  1.000005e+00  ...   30446.999458       4.321120
min   -1.728340e+00 -1.345229e+00  ...       1.000000       0.000000
25%   -8.662007e-01 -1.029760e+00  ...   26368.500000       0.000000
50%   -2.767762e-03  7.998521e-01  ...   52736.000000       0.000000
75%    8.672785e-01  9.008977e-01  ...   79103.500000       0.000000
max    1.734808e+00  1.137185e+00  ...  105471.000000     100.000000

[8 rows x 1790 columns]


In [39]:
df_imputed_normalized.to_csv('train_v2_preprocessed_normalized.csv')